Description:

In [1]:
import pandas as pd
from geofunctions import utils
import geopandas as gpd

In [2]:
import importlib

importlib.reload(utils)

<module 'geofunctions.utils' from '/Users/inessat/Documents/phd/empirics/israel/notebooks/phd-remote-work/geofunctions/utils.py'>

In [3]:
keywords = ["pois"]
dir_path = utils.get_path("raw", "osm", "israel-and-palestine-latest-free.shp")

In [4]:
import os

list_files = []
for file in os.listdir(dir_path):
    if str.endswith(file, "shp") and len([i for i in keywords if i in file]) > 0:
        print(file)
        df = gpd.read_file(os.path.join(dir_path, file))
        df["file"] = file
        list_files.append(df)
df_full = pd.concat(list_files)

gis_osm_pois_free_1.shp
gis_osm_pois_a_free_1.shp


In [14]:
df_full.shape

(97728, 6)

In [15]:
df_full.head()

,osm_id,code,fclass,name,geometry,file
0,29944308,2721,attraction,שער האשפות,POINT (35.23417 31.77483),gis_osm_pois_free_1.shp
1,34495605,2701,tourist_info,None,POINT (35.54209 32.78561),gis_osm_pois_free_1.shp
2,114162322,2401,hotel,מלון דניאל,POINT (34.80059 32.17195),gis_osm_pois_free_1.shp
3,114162327,2401,hotel,מלון השרון,POINT (34.80311 32.17897),gis_osm_pois_free_1.shp
4,246270624,2203,cinema,סינמה סיטי גלילות,POINT (34.80471 32.14666),gis_osm_pois_free_1.shp


# Testing for duplicates

In [16]:
df_full.osm_id.value_counts()

osm_id
604139879     3
431624878     3
999306431     2
4679384289    2
498805290     2
             ..
5679966567    1
5679672221    1
5679590023    1
5678946004    1
1341246663    1
Name: count, Length: 97487, dtype: int64

In [ ]:
df_full[df_full.osm_id == "604139879"]

,osm_id,code,fclass,name,geometry,file
20015,604139879,2421,shelter,שומרה,"POLYGON ((35.08865 31.81091, 35.08865 31.81091...",gis_osm_pois_a_free_1.shp
20016,604139879,2953,observation_tower,שומרה,"POLYGON ((35.08865 31.81091, 35.08865 31.81091...",gis_osm_pois_a_free_1.shp
20017,604139879,2742,viewpoint,שומרה,"POLYGON ((35.08865 31.81091, 35.08865 31.81091...",gis_osm_pois_a_free_1.shp


In [ ]:
df_full.drop_duplicates(["osm_id", "fclass"]).shape

(97728, 6)

In [20]:
df_full.shape

(97728, 6)

# Clip by gushdan boundaries

In [5]:
gdf_pois_gushdan = utils.clip_gdf_gushdan_boundaries(df_full, "geometry")

joined
Original shape: 97728, new shape: 30043


# Analyse

In [6]:
gdf_pois_gushdan.fclass.value_counts()

fclass
park                 5575
pitch                2609
playground           2164
bench                1419
school               1336
                     ... 
newsagent               1
vending_cigarette       1
water_mill              1
battlefield             1
fort                    1
Name: count, Length: 131, dtype: int64

In [7]:
gdf_pois_gushdan.fclass.value_counts()[gdf_pois_gushdan.fclass.value_counts() > 100]

fclass
park                5575
pitch               2609
playground          2164
bench               1419
school              1336
drinking_water      1153
restaurant           929
waste_basket         812
kindergarten         784
fast_food            727
supermarket          667
cafe                 650
swimming_pool        584
recycling            455
clothes              452
convenience          438
sports_centre        326
toilet               309
artwork              304
bank                 282
telephone            251
clinic               247
pharmacy             228
bakery               227
atm                  218
hairdresser          216
memorial             215
fountain             204
shelter              198
mall                 197
bicycle_rental       196
graveyard            196
viewpoint            185
community_centre     181
bar                  179
dog_park             177
recycling_paper      171
attraction           165
picnic_site          164
post_office       

### Filtering out categories that are not urban amenities

In [8]:
urban_amenities = {
    "food": ["bar", "cafe", "restaurant", "fast_food", "pub", "food_court"],
    "education": ["school", "university", "college", "kindergarten"],
    "Health": ["hospital", "clinic", "pharmacy", "dentist", "doctors"],
    "entertainment": [
        "cinema",
        "theatre",
        "nightclub",
        "arts_centre",
        "community_centre",
        "library",
        "museum"
    ],
    "outdoor_attractions": ["attraction", "memorial", "ruins", "artwork", "fountain","archaeological"],
    "parks_sport": [
        "park",  # Parks
        "pitch",  # Sports pitches
        "sports_centre",  # Sports centers
        "stadium",  # Stadiums
        "playground",  # Playgrounds
        "swimming_pool",
        "dog_park",
        "viewpoint"
    ],
    "retail": [
        "supermarket",
        "mall",
        "convenience",
        "clothes",
        "pharmacy",
        "bakery",
        "electronics",
        "furniture",
        "books",
        "shoes",
        "jewelry",
        "sports",
        "toys",
        "optician",
        "stationery",
        "department_store",
        "florist",
        "kiosk",
        "car_dealership"
    ],
    "services": [
        "bank",
        "hairdresser",
        "nails",  # Nail salons
        "spa",  # Spas and wellness centers
        "massage",  # Massage therapy centers
        "tattoo",  # Tattoo parlors
        "photography",  # Photography studios
        "travel_agency",  # Travel agencies
        "bicycle_repair",
        "car_repair",
        "car_wash",
        "post_office",
        "bicycle_rental"
    ],
}

In [96]:
urban_amenities['retail'].extend([ i for i  in gdf_pois_gushdan.fclass if 'shop' in i])

In [9]:
pois_list = [amenity for category in urban_amenities.values() for amenity in category]

In [98]:
gdf_pois_gushdan[
    ~gdf_pois_gushdan.fclass.isin(pois_list)
].fclass.value_counts().head(20)

fclass
bench              1419
drinking_water     1153
waste_basket        812
recycling           455
toilet              309
telephone           251
atm                 218
shelter             198
graveyard           196
recycling_paper     171
picnic_site         164
post_box            139
hotel               136
greengrocer         131
recycling_glass      97
tourist_info         95
doityourself         94
embassy              85
track                74
town_hall            71
Name: count, dtype: int64

In [ ]:
gdf_pois_gushdan_filtered = gdf_pois_gushdan[
    gdf_pois_gushdan.fclass.isin(pois_list)
]
# just in case we process full dataset as well
gdf_pois_filtered = df_full[
    df_full.fclass.isin(pois_list)
]

In [20]:
gdf_pois_filtered.fclass.value_counts()

fclass
park                18277
pitch                7771
playground           5717
school               5370
supermarket          4952
restaurant           2948
swimming_pool        2497
kindergarten         1947
cafe                 1544
viewpoint            1527
fast_food            1497
pharmacy             1490
convenience          1289
clothes              1220
ruins                1122
attraction           1053
archaeological       1035
sports_centre         828
clinic                793
bank                  766
community_centre      738
bakery                738
memorial              693
artwork               639
hairdresser           504
mall                  430
hospital              423
post_office           376
doctors               367
bar                   337
fountain              332
library               304
car_dealership        285
museum                259
car_wash              253
college               248
dog_park              227
dentist               218
bicyc

### Creating column with POIS category 

In [12]:
# Create a mapping of amenities to categories
category_mapping = {item: category for category, items in urban_amenities.items() for item in items}

In [22]:
category_mapping['supermarket']

'retail'

In [15]:

gdf_pois_gushdan_filtered["pois_category"] = gdf_pois_gushdan_filtered.fclass.map(category_mapping)
gdf_pois_filtered["pois_category"] = gdf_pois_filtered.fclass.map(category_mapping)

In [27]:
gdf_pois_filtered["pois_category"].value_counts()

pois_category
parks_sport            36955
retail                 11126
education               7669
food                    6515
outdoor_attractions     4874
services                2112
Health                  1801
entertainment           1724
Name: count, dtype: int64

In [19]:
gdf_pois_gushdan_filtered["pois_category"].value_counts()

pois_category
parks_sport            11650
retail                  2605
food                    2585
education               2179
outdoor_attractions     1129
services                 940
entertainment            519
Health                   358
Name: count, dtype: int64

In [107]:
gdf_pois_gushdan_filtered.head()

,osm_id,code,fclass,name,geometry,file,index_right,pois_category
4,246270624,2203,cinema,סינמה סיטי גלילות,POINT (34.80471 32.14666),gis_osm_pois_free_1.shp,0,entertainment
17,268157256,2101,pharmacy,Super-Pharm,POINT (34.85329 32.18437),gis_osm_pois_free_1.shp,0,retail
19,268162810,2084,college,מכללת נתניה,POINT (34.87984 32.30745),gis_osm_pois_free_1.shp,0,education
20,268162811,2601,bank,בנק הפועלים,POINT (34.87502 32.30774),gis_osm_pois_free_1.shp,0,services
21,268162812,2101,pharmacy,None,POINT (34.87054 32.31196),gis_osm_pois_free_1.shp,0,retail


In [112]:
gdf_pois_gushdan_filtered=gdf_pois_gushdan_filtered.drop("index_right", axis=1)

In [109]:
gdf_pois_gushdan_filtered[gdf_pois_gushdan_filtered.file=='gis_osm_pois_a_free_1.shp'].geometry.geom_type.value_counts()

Polygon         13519
MultiPolygon       14
Name: count, dtype: int64

In [110]:
gdf_pois_gushdan_filtered[gdf_pois_gushdan_filtered.file=='gis_osm_pois_free_1.shp'].geometry.geom_type.value_counts()

Point    9135
Name: count, dtype: int64

POIS are interesected but it makes sense

In [113]:
gdf_pois_gushdan_filtered[gdf_pois_gushdan_filtered.file=='gis_osm_pois_free_1.shp'].sjoin(gdf_pois_gushdan_filtered[gdf_pois_gushdan_filtered.file=='gis_osm_pois_a_free_1.shp']
                                                                                           ,how='inner' )

,osm_id_left,code_left,fclass_left,name_left,geometry,file_left,pois_category_left,index_right,osm_id_right,code_right,fclass_right,name_right,file_right,pois_category_right
4,246270624,2203,cinema,סינמה סיטי גלילות,POINT (34.80471 32.14666),gis_osm_pois_free_1.shp,entertainment,213,37986452,2504,mall,רב מכר גלילות,gis_osm_pois_a_free_1.shp,retail
17,268157256,2101,pharmacy,Super-Pharm,POINT (34.85329 32.18437),gis_osm_pois_free_1.shp,retail,2304,102716891,2504,mall,קניון פארק,gis_osm_pois_a_free_1.shp,retail
25,270161934,2203,cinema,סינמה פארק רעננה,POINT (34.85312 32.18403),gis_osm_pois_free_1.shp,entertainment,2304,102716891,2504,mall,קניון פארק,gis_osm_pois_a_free_1.shp,retail
43,275589899,2082,school,בית הספר החקלאי מקווה ישראל,POINT (34.7811 32.02926),gis_osm_pois_free_1.shp,education,32430,1004703155,2082,school,בית הספר החקלאי מקווה ישראל,gis_osm_pois_a_free_1.shp,education
172,278473935,2724,memorial,קבר וייצמן,POINT (34.81915 31.9064),gis_osm_pois_free_1.shp,outdoor_attractions,18806,555836234,2081,university,מכון ויצמן למדע,gis_osm_pois_a_free_1.shp,education
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52017,12290189857,2904,fountain,None,POINT (34.81185 32.10136),gis_osm_pois_free_1.shp,outdoor_attractions,41399,16022802,2204,park,גני יהושע,gis_osm_pois_a_free_1.shp,parks_sport
52074,12325527901,2721,attraction,עגלת קפה - בוטקה,POINT (34.96855 31.88879),gis_osm_pois_free_1.shp,outdoor_attractions,18628,553272319,2204,park,None,gis_osm_pois_a_free_1.shp,parks_sport
52075,12325527902,2303,cafe,עגלת קפה השניים,POINT (34.94503 32.00516),gis_osm_pois_free_1.shp,food,4476,181032395,2204,park,גן החבל,gis_osm_pois_a_free_1.shp,parks_sport
52330,12393444326,2111,clinic,מכבי,POINT (34.70463 31.79535),gis_osm_pois_free_1.shp,Health,11891,368780364,2504,mall,פרנדלי גן יבנה,gis_osm_pois_a_free_1.shp,retail


In [115]:
gdf_pois_gushdan_filtered = gdf_pois_gushdan_filtered.drop(["file"],axis=1)

In [116]:
gdf_pois_gushdan_filtered.to_file(utils.get_path("processed","osm","osm_pois.geojson"))

In [31]:
gdf_pois_filtered.drop("geometry",axis=1).to_file(utils.get_path("processed","osm","osm_pois_israell.geojson"))